<a href="https://colab.research.google.com/github/epalosh/SEES-2022/blob/main/Ocean_Surface_Temperature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ocean Surface Temperatures Using GEE!

*Notebook by: Ethan Palosh*

Dataset: "NOAA AVHRR Pathfinder Version 5.3 Collated Global 4km Sea Surface Temperature" 
https://developers.google.com/earth-engine/datasets/catalog/NOAA_CDR_SST_PATHFINDER_V53?hl=en#bands 

# Earth Engine Setup

In [78]:
!pip install geemap &> /dev/null
!pip install mss &> /dev/null

To import GEE:


1.   Run the following code cell
2.   Click link and copy verification
3.   Paste in text box and hit "enter"





In [2]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=CqtW5pqna5S5xUKpWBcp7Ao8xWXcZeDPBc830UeDhWc&tc=-piqjE4UbxM1fF__yq1MVoqSUt3GnCly1ZwMHzJMuOk&cc=MyQ6X6msZr5JAHoMOYuZycAp3eJgDMMDN9MWL-Po68Q

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdQt8qj0fxWLDFnZ6uQR-8T4SjIx2ubItCjBOkZCwbRfeL1EGptpFoZWu2Y

Successfully saved authorization token.


In [ ]:
import os

download_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
html_file = os.path.join(download_dir, 'my_map.html')

### GEE Map setup (for interactive map display)

In [77]:
# Installs geemap package
import geemap.timelapse as timelapse
import subprocess

try:
    import geemap as geemap
    print("geemap is imported and ready to use in Colab")
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    import geemap as geemap
    print("geemap is now installed, imported and ready to use in Colab") 

#geemap.update_package()

geemap is imported and ready to use in Colab


**Folium import:**
Commented out for this notebook!

In [5]:
# import folium


# def add_ee_layer(self, ee_image_object, vis_params, name):
#   map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
#   folium.raster_layers.TileLayer(
#       tiles=map_id_dict['tile_fetcher'].url_format,
#       attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
#       name=name,
#       overlay=True,
#       control=True
#   ).add_to(self)

# folium.Map.add_ee_layer = add_ee_layer

#Static Maps: Difference From Average (El Nino, La Nina)

**Next, pull data from the database you plan to analyze using a date range.** 

Here, I get a target range for an El Nino season. (2015-2016) On the next line, I pull a 10 year range of data before the anomaly to compare it to.

The variables are named accordingly, each storing an "image collection."

In [62]:
# Creates the image collection
noaaEN = ee.ImageCollection("NOAA/CDR/SST_PATHFINDER/V53").filterDate("2015-06-01", "2016-06-01")
tenYrAvgEN = ee.ImageCollection("NOAA/CDR/SST_PATHFINDER/V53").filterDate("2004-01-01", "2014-01-01") 

If you plan to bound the data, you can define these bounds and clip images with it later:

In [63]:
# Select the bounds of the area to be clipped
# bounds = ee.Geometry.Point([43.4210, -47.7686]).buffer(40)

**Next, apply a reducer to the image collections to produce a single image:**

*   "mean" takes an average of surface temperature over the 10 years
*   "EN" (short for El Nino) averages the anomaly years to a single image
*   "diff" produces an image product of the difference between both of the images






In [64]:
# Selects the image from the image collection
mean = tenYrAvgEN.mean()#.clip(bounds)
EN = noaaEN.mean()
diffEN = mean.subtract(EN)

**Next, I define a palette and two dictionaries of parameters to be used for mapping.**

*keyParams was created to display the correct unit scale on the map. (The raw data is at a scale factor of 0.01)*

In [65]:
palette = ["#00BCE3", "blue", "#00008B", "green", "yellow", "orange", "red"]

# Choosing the band (Daily Sea Surface Temperature), max/min, and palette
mapParams = {
    "bands": ["sea_surface_temperature"],
    "min": -700, #scale factor of .01
    "max": 700,
    "palette": palette
}

keyParams = {
    "min": -7,
    "max": 7,
    "palette": palette
}

**Finally, map it!**

In [66]:
# Display the map
Map = geemap.Map()
Map.addLayer(diffEN, mapParams, "Ocean Temperature")
# Map.centerObject(bounds, 7)
Map.add_colorbar(vis_params=keyParams, label='Difference From 10 Year Average Surface Temperature (°C)', position='bottomright')

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

**Next, we can map a similar map that investigates a La Nina event.**

Start by creating image collections, defining date ranges, and reducing the collections to single images.

In [74]:
# Creates the image collection
noaaLN = ee.ImageCollection("NOAA/CDR/SST_PATHFINDER/V53").filterDate("1998-06-01", "1999-06-01")
tenYrAvgLN = ee.ImageCollection("NOAA/CDR/SST_PATHFINDER/V53").filterDate("1987-01-01", "1997-01-01") 

# Selects the image from the image collection
mean = tenYrAvgLN.mean()#.clip(bounds)
LN = noaaLN.mean()
diffLN = mean.subtract(LN)

**Then map it using parameters we already defined!**

In [75]:
# Display the map
Map = geemap.Map()
Map.addLayer(diffLN, mapParams, "Ocean Temperature")
# Map.centerObject(bounds, 7)
Map.add_colorbar(vis_params=keyParams, label='Difference From 10 Year Average Surface Temperature (°C)', position='bottomright')

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

#GIF (Ocean Surface Temperature)

**In this section, I create a GIF that shows ocean surface temp monthly over 3 years as recorded by Pathfinder.** 

The primary step is to call the "create_timelapse" function, and populate the arguments with data we've already defined. There are more arguments to mess with if you choose, but not all are listed here. 

In [68]:
gif = timelapse.create_timelapse(tenYrAvgEN, "2010-03-01", "2013-12-31", \
                                 frequency="month", \
                                 reducer='median', \
                                 palette=palette, \
                                 region=ee.Geometry.Polygon([[-250, 50], [-250, -50], [-80, -50], [-80, 50]]), \
                                 bands="sea_surface_temperature", \
                                 dimensions=768, \
                                 font_size=35, \
                                 add_colorbar=True, \
                                 colorbar_orientation='horizontal', 
                                 )

Generating URL...
Please wait ...
The GIF image has been saved to: /tmp/094147ab-6953-4028-aa97-0d6309c8c391.gif


Finally, you may choose to convert to an mp4 using the function below.

In [69]:
timelapse.gif_to_mp4(gif, "gif_mp4")